In [ ]:
# Setup: Install Gradio
!pip install gradio --quiet

import gradio as gr

# Store session data for each user (simulated context)
user_state = {}

# Simulated language model response logic
def handle_user_input(message, state):
    message = message.lower()

    if "schedule" in message:
        return "Sure, how long should the meeting be?", {"intent": "initiate_scheduling"}
    elif "hour" in message:
        state["duration"] = "1 hour"
        return "Understood. Do you have a preferred day or time?", state
    elif "tuesday" in message:
        state["preferred_time"] = "Tuesday afternoon"
        return "I can book you at 2:00 PM or 4:30 PM. Which one would you prefer?", state
    else:
        return "Could you clarify when you'd like to meet?", state

# Simulated calendar system
def connect_to_calendar():
    return "mock_auth_token"

def fetch_available_times(auth_token, length, time_slot):
    return ["2:00 PM", "4:30 PM"]

# Gradio-compatible chat handler
def chat_logic(user_input, history):
    try:
        session_id = "user123"
        if session_id not in user_state:
            user_state[session_id] = {"context": {}}

        current_context = user_state[session_id]["context"]
        reply, updated_context = handle_user_input(user_input, current_context)
        user_state[session_id]["context"] = updated_context

        if updated_context.get("duration") and updated_context.get("preferred_time"):
            token = connect_to_calendar()
            times = fetch_available_times(token, updated_context["duration"], updated_context["preferred_time"])
            reply += f"\nHere are the open slots: {', '.join(times)}"

        history.append((user_input, reply))
        return "", history

    except Exception as error:
        history.append(("SYSTEM", f"⚠️ An unexpected error occurred: {error}"))
        return "", history

# Launching a Gradio chat interface using Blocks API
with gr.Blocks() as ui:
    gr.Markdown("## 📅 Smart Scheduler - AI Assistant")
    conversation = gr.Chatbot()
    user_box = gr.Textbox(placeholder="Ask me to schedule a meeting...")
    reset_button = gr.Button("Clear")

    user_box.submit(chat_logic, [user_box, conversation], [user_box, conversation])
    reset_button.click(lambda: [], None, conversation, queue=False)

ui.launch(share=True)



<ipython-input-9-3389215683>:57: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  conversation = gr.Chatbot()


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://824b60399240e7933a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
